In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import nibabel as nib
import pcntoolkit as ptk 
from pcntoolkit.dataio.fileio import load as ptkload
from pcntoolkit.dataio.fileio import save as ptksave
from pcntoolkit.dataio.fileio import save_nifti, load_nifti
from pcntoolkit.normative import predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix, calibration_descriptives

from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
import pingouin as pg

/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#Define:
def compute_pearsonr(A, B):
    """ Manually computes the Pearson correlation between two matrices.
        Basic usage::
            compute_pearsonr(A, B)
        :param A: an N * M data array
        :param cov: an N * M array
        :returns Rho: N dimensional vector of correlation coefficients
        :returns ys2: N dimensional vector of p-values
        Notes::
            This function is useful when M is large and only the diagonal entries
            of the resulting correlation matrix are of interest. This function
            does not compute the full correlation matrix as an intermediate step"""

    # N = A.shape[1]
    N = A.shape[0]

    # first mean centre
    Am = A - np.mean(A, axis=0)
    Bm = B - np.mean(B, axis=0)
    # then normalize
    An = Am / np.sqrt(np.sum(Am**2, axis=0))
    Bn = Bm / np.sqrt(np.sum(Bm**2, axis=0))
    del(Am, Bm)

    Rho = np.sum(An * Bn, axis=0)
    del(An, Bn)

    # Fisher r-to-z
    Zr = (np.arctanh(Rho) - np.arctanh(0)) * np.sqrt(N - 3)
    N = stats.norm()
    pRho = 2*N.cdf(-np.abs(Zr))
    # pRho = 1-N.cdf(Zr)
    
    return Rho, pRho

 #%%  
 # globals
root_dir = '/project_cephfs/3022017.06/ENIGMA_ANX/'

###  CHANGE DEPENDING ON Z-STAT OR SCALED EFFECT  ###
proc_dir = os.path.join(root_dir,'Z_stat/')
#proc_dir = os.path.join(root_dir,'Scaled_effect/')

data_dir = os.path.join(proc_dir,'data/')
w_dir = os.path.join(proc_dir,'vox/')
struc_coef_dir = os.path.join(w_dir, 'Structure_Coef/')
mask_nii = ('/opt/fmriprep/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz')
ex_nii = os.path.join(data_dir, 'ENIGMA_FC_tr_1.nii.gz')

In [3]:
# load covariatea
print('loading covariate data ...')
df_dem = pd.read_csv(os.path.join(data_dir,'metadata_te.csv'))

#load the pkl file
yhat_est = ptkload(os.path.join(w_dir,'yhat_estimate.pkl'), mask=mask_nii)
yhat_est_transf = np.transpose(yhat_est)
cols_cov = ["Age", 
            "Sex",
            "MRI", 
            "Instructions",
            "Precond_number_trials",
            "Multiple_CSplus", 
            "Multiple_CSminus",
            "CS_type_neutral_faces",
            "CS_type_neutral_pictures",
            "CS_type_neutral_male_avatar",
            "CS_type_snakes_spiders",
            "CS_type_gabor_patch",
            "CS_type_animal_tool",
            "CS_type_affective_faces_pictures",
            "CS_type_humanoic_characters",
            "Number_CSplus_cond",
            "Number_CSminus_cond",
            "Reinforcing_rate",
            "US_type_electric_shock", 
            "US_type_auditory", 
            "US_type_visceral",
            "US_type_thermal", 
            "Average_ITI", 
            "Average_ISI",
            "Potential_US_confound"] 

loading covariate data ...


In [5]:
#For each covariate of interest [n x 1]
for column in cols_cov:
    curr_cov = df_dem[column]
    curr_cov = curr_cov.astype(int)
    print(curr_cov)
    
    covariate_Rho = []
    covariate_pRho = []
    coviariate_Rho_Sqrd = []
    
    covariate_Rho_array = np.empty(0)
    covariate_pRho_array = np.empty(0)
    coviariate_Rho_Sqrd_array = np.empty(0)    
    
    #For each voxel of the brain [n x 1]
    for voxel in range(0, yhat_est.shape[1]):
        #Correlate the covariate with the predicted activity in that voxel [1 x 1 - with n data points]
        correlation = compute_pearsonr(curr_cov, yhat_est[:,voxel])
        #Accumulate the correlations to form one whole brain vector of 
        # vector of correlation coefficients
        covariate_Rho.append(correlation[0]) 
        coviariate_Rho_Sqrd.append(correlation[0]*correlation[0]) 
        #vector of p-values
        covariate_pRho.append(correlation[1]) 
        
    #Convert the lists to arrays
    covariate_Rho_array = np.array(covariate_Rho)
    coviariate_Rho_Sqrd_array = np.array(coviariate_Rho_Sqrd)
    covariate_pRho_array = np.array(covariate_pRho)
      
    #Build the filename for each output file    
    filename_Rho = (struc_coef_dir +'Rho_' +column +'.nii.gz')
    filename_Rho_Sqrd = (struc_coef_dir +'Rho_Sqrd_' +column +'.nii.gz')  
    filename_pRho = (struc_coef_dir +'pRho_' +column +'.nii.gz')
       
    #Save each output file as a nii
    save_nifti(covariate_Rho_array, filename_Rho, mask = mask_nii, examplenii = mask_nii, dtype='float32')
    save_nifti(coviariate_Rho_Sqrd_array, filename_Rho_Sqrd, mask = mask_nii, examplenii = mask_nii, dtype='float32')
    save_nifti(covariate_pRho_array, filename_pRho, mask = mask_nii, examplenii = mask_nii, dtype='float32')

print('DONE')

0      24
1      22
2      21
3      18
4      26
       ..
641    36
642    20
643    26
644    26
645    32
Name: Age, Length: 646, dtype: int64
0      0
1      1
2      1
3      0
4      1
      ..
641    1
642    0
643    0
644    0
645    1
Name: Sex, Length: 646, dtype: int64
0      1
1      1
2      1
3      1
4      1
      ..
641    1
642    1
643    1
644    1
645    1
Name: MRI, Length: 646, dtype: int64
0      1
1      1
2      1
3      1
4      1
      ..
641    1
642    1
643    1
644    1
645    1
Name: Instructions, Length: 646, dtype: int64
0      8
1      8
2      8
3      8
4      8
      ..
641    8
642    8
643    8
644    8
645    8
Name: Precond_number_trials, Length: 646, dtype: int64
0      1
1      1
2      1
3      1
4      1
      ..
641    0
642    0
643    0
644    0
645    0
Name: Multiple_CSplus, Length: 646, dtype: int64
0      1
1      1
2      1
3      1
4      1
      ..
641    0
642    0
643    0
644    0
645    0
Name: Multiple_CSminus, Length: 646

In [6]:
#%% Threshold R2 <0.3   
out_dir = os.path.join(w_dir, 'Structure_Coef/')
for column in cols_cov:
    curr_cov = df_dem[column]
    print(column)
    
    in_filename = (out_dir +'Rho_Sqrd_' +column +'.nii.gz')  
    
    out_filename = (out_dir +'Rho_Sqrd_' +column +'_gt0pt3.nii.gz')

    #print(out_filename_pos, out_filename_neg)
    
    command = ('fslmaths ' +in_filename +' -thr 0.3 ' +out_filename)
    print(command)
    os.system(command)
    !command

print('DONE')

Age
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Age.nii.gz -thr 0.3 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Age_gt0pt3.nii.gz
Sex
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Sex.nii.gz -thr 0.3 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Sex_gt0pt3.nii.gz
MRI
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_MRI.nii.gz -thr 0.3 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_MRI_gt0pt3.nii.gz
Instructions
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Instructions.nii.gz -thr 0.3 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Instructions_gt0pt3.nii.gz
Precond_number_trials
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Precond_number_trials.nii.gz -thr 0.3 /project_cephfs/3022017.06/ENIGMA_ANX/Z_sta

In [7]:
#%% Mask Rho with regions only greater than R2 <0.3 
for column in cols_cov:
    curr_cov = df_dem[column]
    print(column)
 
    in_filename = (out_dir +'Rho_' +column +'.nii.gz')  
    out_filename = (out_dir +'Rho_' +column +'_thr_r2_03.nii.gz')
    mask_filename = (out_dir +'Rho_Sqrd_' +column +'_gt0pt3.nii.gz')
    
    command = ('fslmaths ' +in_filename +' -mas ' +mask_filename +' ' +out_filename)
    print(command)
    os.system(command)
    !command

print('DONE')

Age
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Age.nii.gz -mas /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Age_gt0pt3.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Age_thr_r2_03.nii.gz
Sex
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sex.nii.gz -mas /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_Sex_gt0pt3.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sex_thr_r2_03.nii.gz
MRI
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_MRI.nii.gz -mas /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Sqrd_MRI_gt0pt3.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_MRI_thr_r2_03.nii.gz
Instructions
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Structure_Coef/Rho_Instructions.nii.gz -mas /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Struc